In [6]:
import pandas as pd
from dbfread import DBF
#修改属性值
path = 'D:\\IPMIProject\\master test\\20231223.dbf' # 文件目录
table = DBF(path, encoding='GBK')
df = pd.DataFrame(iter(table))
# print(df.columns)
allowed_values1 = ['101','109','199']
allowed_values2 = ['201','202','203','204']
allowed_values3 = ['301','302','303','306','307','311','312','313','314','315','333','396','397','398']
allowed_values4 = ['408','414','415','497','498']
df.loc[~df['ZZLLM'].isin(allowed_values1), 'ZZLLM'] = '000'
df.loc[~df['WGYM'].isin(allowed_values2), 'WGYM'] = '000'
df.loc[~df['YWK1M'].isin(allowed_values3), 'YWK1M'] = '000'
df.loc[~df['YWK2M'].isin(allowed_values4), 'YWK2M'] = '000'
# print(df)
df.columns = df.columns.map(str.lower)

#对修改后的值进行分类
df1 = df[["bmh","zzllm","wgym","ywk1m","ywk2m","zzllkddm"]]
# print(df1)

# 使用其中的四个属性进行分组
df2 = df1.groupby(["zzllm","wgym","ywk1m","ywk2m","zzllkddm"]).count()
print(df2)
df2.fillna(method='bfill', inplace=True)
print(df2)
df2.to_excel("classfier.xlsx")
print(df2['bmh'].sum())

                                 bmh
zzllm wgym ywk1m ywk2m zzllkddm     
101   000  000   000   6103      112
                       6105       67
                       6107       55
                       6111      111
                       6112      118
...                              ...
199   204  000   000   6156      297
                       6157      479
                       6158      162
                       6159       75
                       6160      276

[1237 rows x 1 columns]
                                 bmh
zzllm wgym ywk1m ywk2m zzllkddm     
101   000  000   000   6103      112
                       6105       67
                       6107       55
                       6111      111
                       6112      118
...                              ...
199   204  000   000   6156      297
                       6157      479
                       6158      162
                       6159       75
                       6160      276

[1237 rows x

In [7]:
# 读取数据，并按照zzllkddm进行分组
path = 'D:\\IPMIProject\\master test\\20231223.dbf' # 文件目录
table = DBF(path, encoding='GBK')
arr = pd.DataFrame(iter(table))
allowed_values1 = ['101','109','199']
allowed_values2 = ['201','202','203','204']
allowed_values3 = ['301','302','303','306','307','311','312','313','314','315','333','396','397','398']
allowed_values4 = ['408','414','415','497','498']
arr.loc[~arr['ZZLLM'].isin(allowed_values1), 'ZZLLM'] = '000'
arr.loc[~arr['WGYM'].isin(allowed_values2), 'WGYM'] = '000'
arr.loc[~arr['YWK1M'].isin(allowed_values3), 'YWK1M'] = '000'
arr.loc[~arr['YWK2M'].isin(allowed_values4), 'YWK2M'] = '000'
arr.columns = arr.columns.map(str.lower)

# arr = df[["bmh","zzllm","wgym","ywk1m","ywk2m","zzllkddm"]]
list1 = arr['zzllkddm'].unique()
groups = arr.groupby("zzllkddm")
for i in arr['zzllkddm'].unique():
    groups.get_group(i).to_excel(str(i)+".xlsx")


In [9]:
# 统计每个考点有多少类别，以及每个类别有多少人，参考属性有["zzllmc","wgymc","ywk1mc","ywk2mc","bkdwdm"]

for i in arr['zzllkddm'].unique():
    df1 = pd.read_excel(str(i)+".xlsx")

    # 使用其中的四个属性进行分组,"bkdwdm"
    df2 = df1.groupby(["bmh","zzllm","wgym","ywk1m","ywk2m","zzllkddm"]).count()
    df2.to_excel("classfier"+str(i)+".xlsx")

# 结果里面bmh表示的是每个专业的人数

In [11]:
import numpy as np

# 数据预处理、提取出每个考点，每个专业的人数
sums = 0
# print(list1)
for i in arr['zzllkddm'].unique():
    # .fillna(method="ffill")向下填充
    res = pd.read_excel("classfier"+str(i)+".xlsx").fillna(method="ffill")
    # dataframe按各专业人数一列排序，inplace原地排序，ascending升序
    # res.sort_values(by="bmh",inplace=True,ascending=False)
    # 取出各专业人数一列转为array
    # 若存在控制需要填充
    # res = res.fillna(res['bmh'].mean())
    res["bmh"]=1
    StuNum = res[["bmh","zzllm","wgym","ywk1m","ywk2m","bkdwdm"]]
    # StuNum.groupby(["zzllm","wgym","ywk1m","ywk2m","bkdwdm"]).count()
    # sums += res['bmh'].sum()
    result = StuNum.groupby(["zzllm","wgym","ywk1m","ywk2m","bkdwdm"]).agg(  
        # value=('value', 'sum'),  # 对'value'列使用'sum'聚合  
    # 对'info'列使用'first'聚合（或'last'、'min'、'max'等）  
        count=('bmh', 'size')  # 使用'size'来计算每个组的行数，并将其作为'count'列  
    ).reset_index()
    np.savetxt("classfier"+str(i)+".txt",StuNum,fmt="%d,%s,%s,%s,%s,%s",delimiter=" ")

print(sums)

0


In [ ]:
# 统计每个考点教室所能容纳的人数
groups1 = pd.read_excel("考场容量表.xls").groupby("zzllkddm")
print(groups1.apply(lambda x:x))
print(groups1.get_group(6103)["renshu"].tolist())

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
      zzllkddm zzllkcbh  renshu
0         6103        1      30
1         6103       10      30
2         6103      100      30
3         6103      101      30
4         6103      102      30
...        ...      ...     ...
5420      6160      103      30
5421      6160      104      30
5422      6160      105      30
5423      6160      106      30
5424      6160      107      16

[5425 rows x 3 columns]
[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 29, 30, 30, 30, 31, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 26, 3

In [ ]:
def Sum_matrix(matrix):
    sum=0
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            sum+=matrix[i][j]
    return sum

In [ ]:
import pandas as pd
pd.options.display.max_rows = None
# 分组算法
# pd.set_option('display.max_rows',200)
# StuNum = np.loadtxt("classfier6127.txt")
aa=pd.read_csv("classfier6105.txt",encoding='gb2312',header=None)
aa.columns=['num','s1','s2','s3','s4','s5']
# 对txt文件的数据排序，方便后面索引对应
print("各专业人数")
print(aa.sort_values(by='num',axis=0).reset_index(drop=True))
StuNum = pd.read_csv("classfier6105.txt",encoding='gb2312',header=None).iloc[:,0].tolist()

# 设置参数：教室最多可容纳人数、考场允许最大专业数量
max_student=groups1.get_group(6105)["renshu"].tolist()
print("考场容量：",max_student)
print("考场数量：",len(max_student))
max_subject=5
# 存放分组结果
groups = []

# 这里还没有考虑取余的情况
for i in range(len(StuNum)):
    StuNum[i] = StuNum[i] % 30

StuNum.sort()
# print(len(StuNum))
start = 0
end = len(StuNum)-1
classroom = 0
while(start<end and classroom<len(max_student)):
    class_subject = []#用来存储人数对应的索引，输出的就是这个，和上面txt文件的输出对应
    class_sum = StuNum[end]#每次循环初始，让sum等于尾指针指向的数
    subject = 1
    class_subject.append(end)
    # 人数不超过教室容量、专业不超过教室容量、专业还未分完
    while(class_sum<max_student[classroom] and subject<max_subject and start<end):
        class_sum = class_sum + StuNum[start]
        class_subject.append(start)
        start = start+1
        subject = subject+1
    end = end-1
    classroom+=1
    groups.append(class_subject)

print(groups)
print(len(groups))
# 下面的数字是不同袋子的编号

In [ ]:
import pandas as pd  
  
# 示例数据  
data = {  
    'category': ['A', 'A', 'B', 'C', 'C', 'C'],  
    'value': [1, 2, 3, 4, 5, 6],  
    'info': ['x', 'x', 'z', 'h', 'h', 'u']  
}  
df = pd.DataFrame(data)  
  
# 使用groupby和agg来合并行，并保留某些列的值，同时添加行数信息  
# 注意：这里我们假设'value'列使用'sum'作为示例（你可以根据需要更改），而'info'列使用'first'  
# 我们还添加了一个新列'count'，它包含每个组的行数  
result = df.groupby(['category','info']).agg(  
    # value=('value', 'sum'),  # 对'value'列使用'sum'聚合  
 # 对'info'列使用'first'聚合（或'last'、'min'、'max'等）  
    count=('category', 'size')  # 使用'size'来计算每个组的行数，并将其作为'count'列  
).reset_index()  # 重置索引，使'category'成为普通列  
  
# 输出结果  
print(result)

  category info  count
0        A    x      2
1        B    z      1
2        C    h      2
3        C    u      1
